使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第十一章第二節 日射に関する地域の区分と日射量等）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import numpy as np
import pandas as pd
from functools import lru_cache

## 第二節　日射に関する地域区分と日射量

### 1．適用範囲

省略

### 2. 引用規格

省略

### 3. 用語の定義

省略

### 4．年間日射量地域区分

省略

### 5．日射量等

省略

## 付録 A 傾斜面における単位面積当たりの平均日射量

### A.1 記号及び単位

省略

### A.2 傾斜面における単位面積当たりの平均日射量の計算方法

$$\large
\displaystyle
I_{s,d,t} = \left\{ \begin{array}{ll}
    I_{D,d,t} + I_{d,d,t} & (I_{D,d,t} \geq 0)\\
    I_{d,d,t} & (I_{D,d,t} < 0)\\
    \end{array}
\right.
$$

In [ ]:
# 傾斜面の単位面積当たりの平均日射量
def get_I_s_d_t(P_alpha, P_beta, df):
    """
    :param P_alpha: 方位角 (°)
    :param P_beta: 傾斜角 (°)
    :param df: load_solrad の返り値
    :return: 傾斜面の単位面積当たりの平均日射量
    """
    I_DN_d_t = df['I_DN'].values / 3.6 * 1000
    I_Sky_d_t = df['I_Sky'].values / 3.6 * 1000
    h_d_t = np.radians(df['h'].values)
    A_d_t = np.radians(df['A'].values * (df['A'].values >= 0) + (df['A'].values + 360.0) * (df['A'].values < 0))

    I_D_d_t = get_I_D_d_t(I_DN_d_t, h_d_t, P_alpha, P_beta, A_d_t)
    I_d_d_t = get_I_d_d_t(I_Sky_d_t, P_beta)

    return I_D_d_t * (I_D_d_t >= 0) + I_d_d_t

$$\large
I_{D,d,t} = I_{DN,d,t} \{sin\ h_{d,t}\ cos\ P_{\beta} + cos\ h_{d,t}\ sin\ P_{\beta}\ cos(P_{\alpha} - A_{d,t})\}
$$

In [ ]:
# 傾斜面の単位面積当たりの直達日射量
def get_I_D_d_t(I_DN_d_t, h_d_t, P_alpha, P_beta, A_d_t):
    return I_DN_d_t * (np.sin(h_d_t) * np.cos(P_beta) + np.cos(h_d_t) * np.sin(P_beta) * np.cos(P_alpha - A_d_t))

$$\large
I_{d,d,t} = I_{Sky,d,t} \frac{(1+cos\ P_{\beta})}{2}
$$

$I_{s,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの平均日射量(W/m<sup>2</sup>)  
$I_{D,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの直達日射量(W/m<sup>2</sup>)  
$I_{d,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの天空放射量(W/m<sup>2</sup>)  
$I_{DN,d,t}$：日付$d$の時刻$t$における法線面直達日射量(W/m<sup>2</sup>)  
$I_{Sky,d,t}$：日付$d$の時刻$t$における水平面天空日射量(W/m<sup>2</sup>)  
$h_{d,t}$：日付$d$の時刻$t$における太陽高度(°)  
$A_{d,t}$：日付$d$の時刻$t$における太陽方位角(°)  
$P_{\alpha}$：傾斜面の方位角(°)  
$P_{\beta}$：傾斜面の傾斜角(°)  

In [ ]:
# 傾斜面の単位面積当たりの天空放射量
def get_I_d_d_t(I_Sky_d_t, P_beta):
    return I_Sky_d_t * ((1.0 + np.cos(P_beta)) / 2.0)

<div style="text-align: center;font-weight: bold;">表　傾斜面の方位角</div>

<table>
<thead>
<tr><th>方位角の範囲</th><th>丸めた値</th><th>方位角の範囲</th><th>丸めた値</th></tr>
</thead>
<tbody>
<tr><th>－15度（345度）から15度</th><td>0度</td><th>165度から195度</th><td>180度</td></tr>
<tr><th>15度から45度</th><td>30度</td><th>195度から225度</th><td>210度</td></tr>
<tr><th>45度から75度</th><td>60度</td><th>225度から255度</th><td>240度</td></tr>
<tr><th>75度から105度</th><td>90度</td><th>255度から285度</th><td>270度</td></tr>
<tr><th>105度から135度</th><td>120度</td><th>285度から315度</th><td>300度</td></tr>
<tr><th>135度から165度</th><td>150度</td><th>315度から345度</th><td>330度</td></tr>
</tbody>
</table>

傾斜面の傾斜角$P_{\beta}$は、水平面からの傾斜面の角度であり、傾斜面の傾斜角の一の位を四捨五入した値とし、90度を超える場合は90度とする。